## Build model

In [1]:
import fasttext

In [2]:
!curl https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz --output cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz
!head -n 12404 cooking.stackexchange.txt > cooking.train
!tail -n 3000 cooking.stackexchange.txt > cooking.valid

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  446k  100  446k    0     0   445k      0  0:00:01  0:00:01 --:--:--  445k
x cooking.stackexchange.id
x cooking.stackexchange.txt
x readme.txt


In [3]:
model = fasttext.train_supervised(input="cooking.train")

In [4]:
labels = model.labels
k = len(labels)
labels_indices = [i for i in range(k)]

In [5]:
def predict_proba(model, text_list):
    return [k.tolist() for k in model.predict(["hi", "hello"], k=k)[1]], labels, labels_indices 

In [6]:
predict_proba(model, ["hi", "hello"])

([[0.21386487782001495,
   0.13876928389072418,
   0.08310910314321518,
   0.05838950723409653,
   0.056946080178022385,
   0.029094485566020012,
   0.0248425155878067,
   0.017698852345347404,
   0.017663955688476562,
   0.016023140400648117,
   0.015842197462916374,
   0.01384671963751316,
   0.013841853477060795,
   0.013492103666067123,
   0.012877251021564007,
   0.01134240347892046,
   0.010514877736568451,
   0.008600137196481228,
   0.008135510608553886,
   0.008033995516598225,
   0.0077551887370646,
   0.006925241556018591,
   0.006574397440999746,
   0.006501240190118551,
   0.006404141895473003,
   0.006123505067080259,
   0.005925947800278664,
   0.0057566165924072266,
   0.005545685533434153,
   0.005440066568553448,
   0.0053683058358728886,
   0.005140202585607767,
   0.005127125885337591,
   0.0048130289651453495,
   0.004579639062285423,
   0.004555821884423494,
   0.0044122738763689995,
   0.004105721600353718,
   0.0035606264136731625,
   0.0034627511631697416,
   0

# Unbox

### Pack model

In [7]:
import unboxapi
client = unboxapi.UnboxClient()

User is not logged in.


In [8]:
saved_path = client.pack_model(
    function=predict_proba, 
    model=model,
    model_name="FastModel",
    model_type="fasttext"
)

[2021-05-12 14:41:28,700] INFO - BentoService bundle 'FastModel:20210512144127_3235FB' saved to: /Users/gbayomi/bentoml/repository/FastModel/20210512144127_3235FB


### Load

In [9]:
import bentoml
from pandas import DataFrame

In [10]:
bento_model = bentoml.load(saved_path)

[2021-05-12 14:41:29,269] WARNING - Module `template_model` already loaded, using existing imported module.
[2021-05-12 14:41:29,289] WARNING - pip package requirement fasttext already exist


### Test locally

In [11]:
bento_model.predict([{"text": "great"}])

[{'__label__baking': 0.21386487782001495,
  '__label__food-safety': 0.13876928389072418,
  '__label__substitutions': 0.08310910314321518,
  '__label__equipment': 0.05838950723409653,
  '__label__bread': 0.056946080178022385,
  '__label__chicken': 0.029094485566020012,
  '__label__storage-method': 0.0248425155878067,
  '__label__eggs': 0.017698852345347404,
  '__label__meat': 0.017663955688476562,
  '__label__sauce': 0.016023140400648117,
  '__label__cake': 0.015842197462916374,
  '__label__flavor': 0.01384671963751316,
  '__label__freezing': 0.013841853477060795,
  '__label__storage-lifetime': 0.013492103666067123,
  '__label__vegetables': 0.012877251021564007,
  '__label__cheese': 0.01134240347892046,
  '__label__frying': 0.010514877736568451,
  '__label__coffee': 0.008600137196481228,
  '__label__chocolate': 0.008135510608553886,
  '__label__oil': 0.008033995516598225,
  '__label__oven': 0.0077551887370646,
  '__label__dough': 0.006925241556018591,
  '__label__food-science': 0.006574

In [12]:
bento_model.predictbatch([{"batch": ["great", "terrible"]}])

[[{'__label__baking': 0.21386487782001495,
   '__label__food-safety': 0.13876928389072418,
   '__label__substitutions': 0.08310910314321518,
   '__label__equipment': 0.05838950723409653,
   '__label__bread': 0.056946080178022385,
   '__label__chicken': 0.029094485566020012,
   '__label__storage-method': 0.0248425155878067,
   '__label__eggs': 0.017698852345347404,
   '__label__meat': 0.017663955688476562,
   '__label__sauce': 0.016023140400648117,
   '__label__cake': 0.015842197462916374,
   '__label__flavor': 0.01384671963751316,
   '__label__freezing': 0.013841853477060795,
   '__label__storage-lifetime': 0.013492103666067123,
   '__label__vegetables': 0.012877251021564007,
   '__label__cheese': 0.01134240347892046,
   '__label__frying': 0.010514877736568451,
   '__label__coffee': 0.008600137196481228,
   '__label__chocolate': 0.008135510608553886,
   '__label__oil': 0.008033995516598225,
   '__label__oven': 0.0077551887370646,
   '__label__dough': 0.006925241556018591,
   '__label__

In [13]:
bento_model.predictactive([{"batch": ["great", "terrible"], "n_instances": 2}])

[([0, 1], ['great', 'terrible'])]

### Test as an endpoint

In [14]:
!bentoml run FastModel:latest predict --input '{"text": "Which baking dish is best to bake a banana bread ?"}'

[2021-05-12 14:41:31,085] INFO - Getting latest version FastModel:20210512144127_3235FB
[2021-05-12 14:41:32,606] INFO - {'service_name': 'FastModel', 'service_version': '20210512144127_3235FB', 'api': 'predict', 'task': {'data': '{"text": "Which baking dish is best to bake a banana bread ?"}', 'task_id': '006b9b82-3888-4a4d-a544-8eedf8582737', 'cli_args': ('--input', '{"text": "Which baking dish is best to bake a banana bread ?"}'), 'inference_job_args': {}}, 'result': {'data': '{"__label__baking": 0.21386487782001495, "__label__food-safety": 0.13876928389072418, "__label__substitutions": 0.08310910314321518, "__label__equipment": 0.05838950723409653, "__label__bread": 0.056946080178022385, "__label__chicken": 0.029094485566020012, "__label__storage-method": 0.0248425155878067, "__label__eggs": 0.017698852345347404, "__label__meat": 0.017663955688476562, "__label__sauce": 0.016023140400648117, "__label__cake": 0.015842197462916374, "__label__flavor": 0.01384671963751316, "__label__fre

In [15]:
!bentoml run FastModel:latest predictbatch --input '{"batch": ["great", "terrible"]}'

[2021-05-12 14:41:34,601] INFO - Getting latest version FastModel:20210512144127_3235FB
[2021-05-12 14:41:36,216] INFO - {'service_name': 'FastModel', 'service_version': '20210512144127_3235FB', 'api': 'predictbatch', 'task': {'data': '{"batch": ["great", "terrible"]}', 'task_id': '7869948e-f370-4f5e-9bdd-7230b4806626', 'cli_args': ('--input', '{"batch": ["great", "terrible"]}'), 'inference_job_args': {}}, 'result': {'data': '[{"__label__baking": 0.21386487782001495, "__label__food-safety": 0.13876928389072418, "__label__substitutions": 0.08310910314321518, "__label__equipment": 0.05838950723409653, "__label__bread": 0.056946080178022385, "__label__chicken": 0.029094485566020012, "__label__storage-method": 0.0248425155878067, "__label__eggs": 0.017698852345347404, "__label__meat": 0.017663955688476562, "__label__sauce": 0.016023140400648117, "__label__cake": 0.015842197462916374, "__label__flavor": 0.01384671963751316, "__label__freezing": 0.013841853477060795, "__label__storage-lifeti

In [16]:
!bentoml run FastModel:latest predictactive --input '{"batch": ["great", "terrible"], "n_instances": 2}'

[2021-05-12 14:41:38,297] INFO - Getting latest version FastModel:20210512144127_3235FB
[2021-05-12 14:41:39,837] INFO - {'service_name': 'FastModel', 'service_version': '20210512144127_3235FB', 'api': 'predictactive', 'task': {'data': '{"batch": ["great", "terrible"], "n_instances": 2}', 'task_id': '86197e06-787c-402a-9b33-5335131e87dd', 'cli_args': ('--input', '{"batch": ["great", "terrible"], "n_instances": 2}'), 'inference_job_args': {}}, 'result': {'data': '[[0, 1], ["great", "terrible"]]', 'http_status': 200, 'http_headers': (('Content-Type', 'application/json'),)}, 'request_id': '86197e06-787c-402a-9b33-5335131e87dd'}
[[0, 1], ["great", "terrible"]]
